In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloaderimport Covid19IndiaLoader
from data.processing import get_data

from models.seir.seir_testing import SEIR_Testing
from models.seir.seirhd import SEIRHD
from models.seir.seir_movement import SEIR_Movement
from models.seir.seir_movement_testing import SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from utils.create_report import create_report
from utils.loss import Loss_Calculator
from viz import plot_forecast

## Load Covid19india Data

In [3]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [4]:
predictions_dict = {}

## Select Districts to fit on

In [11]:
districts_to_show = [('Delhi', None)]

## Perform M1 and M2 fits

In [13]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, train_period=7, val_period=7, model=SEIR_Testing,
        data_from_tracker=True, initialisation='intermediate', num_evals=700, 
        which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'],
        smooth_jump=False, smoothing_length=33, smoothing_method='weighted')
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(
        dataframes, state, district, train_period=7, val_period=0, num_evals=700,
        data_from_tracker=False, initialisation='intermediate', model=SEIR_Testing, 
        which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'],
        smooth_jump=True, smoothing_length=33, smoothing_method='weighted')
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
    predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

Performing m1 fit ..
> /home/users/sansiddh/projects/covid/covid-modelling/data/processing.py(39)get_data()
-> if disable_tracker:


(Pdb)  c


> /home/users/sansiddh/projects/covid/covid-modelling/data/processing.py(39)get_data()
-> if disable_tracker:


(Pdb)  c


splitting data ..
splitting data ..
train
     state       date  hospitalised  total_infected  deceased  recovered
32  Delhi 2020-05-28        7852.4         15488.4     316.2     7319.8
33  Delhi 2020-05-29        8462.8         16387.6     344.2     7580.6
34  Delhi 2020-05-30        9250.6         17463.4     381.2     7831.6
35  Delhi 2020-05-31       10025.6         18578.8     425.2     8128.0
36  Delhi 2020-06-01       10798.2         19749.0     473.2     8477.6
val
    state       date hospitalised total_infected deceased recovered
0  Delhi 2020-06-02      11669.2        21000.8    514.8    8816.8
1  Delhi 2020-06-03      12548.8        22291.8    561.6    9181.4
2  Delhi 2020-06-04      13432.4        23589.8    608.6    9548.8
3  Delhi 2020-06-05      14365.2        24953.8    656.2    9932.4
4  Delhi 2020-06-06      15323.6        26314.6    707.4   10283.6
5  Delhi 2020-06-07        16229          27654      761     10664
6  Delhi 2020-06-08        17125          28936    

(Pdb)  q


job exception: 



 87%|████████▋ | 606/700 [00:27<00:04, 22.10trial/s, best loss: 2.9889088958988212]


BdbQuit: 

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [28]:
lc = Loss_Calculator()

In [30]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

train                                    \
                   hospitalised total_infected deceased recovered   
Maharashtra Mumbai            1           0.49     3.55      3.68   

                            val                                    
                   hospitalised total_infected deceased recovered  
Maharashtra Mumbai        14.45           1.38    10.11        36

### M2 Loss DataFrame

In [31]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

train                                  
                   hospitalised total_infected deceased recovered
Maharashtra Mumbai         6.63           1.14     1.13     12.94

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)